In [1]:
import json
from os.path import join
from SPARQLWrapper import SPARQLWrapper, JSON
from string import Template
from tqdm import tqdm

In [2]:
outputFolder = "../static/iiif/"
limit = 99999
offset = 0
suffix = ""

In [3]:
endpoint = "http://localhost:8887/blazegraph/sparql"

In [4]:
sparql = SPARQLWrapper(endpoint)
sparql.setReturnFormat(JSON)

In [5]:
def sparqlResultToDict(results):
    rows = []
    for result in results["results"]["bindings"]:
        row = {}
        for key in list(result.keys()):
            row[key] = result[key]["value"]
        rows.append(row)
    return rows


In [6]:
objectsQuery = """
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX search: <https://platform.swissartresearch.net/search/>
SELECT DISTINCT ?subject ?label WHERE {
    ?subject a search:Object ;
        rdfs:label ?label .
    ?subject crm:P109_has_current_or_former_curator <https://d-nb.info/gnd/1012546-2> .
    FILTER(REGEX(STR(?subject), "zbz-"))
}
"""

In [7]:
sparql.setQuery(objectsQuery)
objects = sparqlResultToDict(sparql.query().convert())

In [8]:
collectionManifest = {
    "@context": "http://iiif.io/api/presentation/3/context.json",
    "id": "https://resource.swissartresearch.net/manifest/zbz-collection" + suffix,
    "type": "Collection",
    "label": {"de": ["Bilder der Schweiz Online"]},
    "items" : []
}

In [9]:
for obj in tqdm(objects[offset:offset + limit]):
    itemManifestUri = obj['subject'].replace('artwork', 'manifest')
    collectionManifest['items'].append({
        "@id": itemManifestUri,
        "type": "Manifest",
        "label": {"de": [ obj['label'] ]}
    })

100%|██████████| 19031/19031 [00:00<00:00, 543230.86it/s]


In [10]:
filename = join(outputFolder, "zbz-collection%s.json" % suffix)
with open(filename, 'w') as f:
    json.dump(collectionManifest, f, ensure_ascii=False, indent=4)